In [13]:
from pymongo import MongoClient
from datetime import datetime

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["MCA_WT"]

# Define collections
customers = db['customers']
products = db['products']
brands = db['brands']
orders = db['orders']

customers.delete_many({})
products.delete_many({})
brands.delete_many({})
orders.delete_many({})

# Sample brand documents
brands_data = [
    {"brand_name": "BrandA", "rating": 4.9},
    {"brand_name": "BrandB", "rating": 4.2},
    {"brand_name": "BrandC", "rating": 4.5},
    {"brand_name": "BrandD", "rating": 4.9},
    {"brand_name": "BrandE", "rating": 4.9}
]

brands.insert_many(brands_data)

# Sample product documents
products_data = [
    {"product_name": "PhoneA", "brand_name": "BrandA", "warranty_period": 1, "rating": 4.6},
    {"product_name": "LaptopB", "brand_name": "BrandB", "warranty_period": 2, "rating": 4.7},
    {"product_name": "TabletC", "brand_name": "BrandC", "warranty_period": 1, "rating": 4.8},
    {"product_name": "CameraD", "brand_name": "BrandD", "warranty_period": 6, "rating": 4.5},
    {"product_name": "HeadphonesE", "brand_name": "BrandE", "warranty_period": 1, "rating": 4.9}
]

products.insert_many(products_data)

# Sample customer documents
customers_data = [
    {"customer_name": "John Doe", "city": "CityA", "contact_number": "1234567890"},
    {"customer_name": "Jane Smith", "city": "CityB", "contact_number": "0987654321"},
    {"customer_name": "Alice Johnson", "city": "CityA", "contact_number": "1357924680"},
    {"customer_name": "David Brown", "city": "CityC", "contact_number": "2468135790"},
    {"customer_name": "Patricia Green", "city": "CityB", "contact_number": "3579246801"}
]

customers.insert_many(customers_data)

# Sample orders documents
orders_data = [
    {"customer_id": customers.find_one({"customer_name": "John Doe"})["_id"], "product_name": "PhoneA", "purchase_date": datetime(2023, 8, 15), "bill_amount": 55000},
    {"customer_id": customers.find_one({"customer_name": "Jane Smith"})["_id"], "product_name": "LaptopB", "purchase_date": datetime(2023, 8, 10), "bill_amount": 45000},
    {"customer_id": customers.find_one({"customer_name": "Alice Johnson"})["_id"], "product_name": "TabletC", "purchase_date": datetime(2023, 8, 15), "bill_amount": 30000},
    {"customer_id": customers.find_one({"customer_name": "David Brown"})["_id"], "product_name": "CameraD", "purchase_date": datetime(2023, 8, 5), "bill_amount": 20000},
    {"customer_id": customers.find_one({"customer_name": "Patricia Green"})["_id"], "product_name": "HeadphonesE", "purchase_date": datetime(2023, 8, 20), "bill_amount": 60000}
]

orders.insert_many(orders_data)


InsertManyResult([ObjectId('672a2e06abed74c341b1ff4c'), ObjectId('672a2e06abed74c341b1ff4d'), ObjectId('672a2e06abed74c341b1ff4e'), ObjectId('672a2e06abed74c341b1ff4f'), ObjectId('672a2e06abed74c341b1ff50')], acknowledged=True)

In [14]:
# List the names of products with a warranty period of one year.

products_with_warranty = products.find({"warranty_period": 1}, {"product_name": 1, "_id": 0})
for product in products_with_warranty:
    print(product['product_name'])

PhoneA
TabletC
HeadphonesE


In [15]:
# List customers who made a purchase on “15/08/2023”.

purchase_date = datetime(2023, 8, 15)
purchases_on_date = orders.find({"purchase_date": purchase_date})



for order in purchases_on_date:
    customer = customers.find_one({"_id": order["customer_id"]}, {"customer_name": 1, "_id": 0})
    print(customer['customer_name'])

John Doe
Alice Johnson


In [27]:
#  Display the names of products with brands that have the highest rating.

# Find the highest brand rating
highest_rating = brands.find_one(sort=[("rating", -1)])

print(highest_rating['rating'])


# Find brands with the highest rating
top_brands = brands.find({"rating": highest_rating['rating']})

# for brand in top_brands:
#     products_with_top_brand = products.find({"brand_name": brand["brand_name"]}, {"product_name": 1, "_id": 0})
#     print("brand : ", brand["brand_name"])
#     for product in products_with_top_brand:
#         print("\t product : ", product["product_name"])


bname = []
for brand in top_brands:
    bname.append(brand["brand_name"])

print(bname)

products_names = products.find({"brand_name" : { "$in" : bname}} , {"product_name": 1, "brand_name":1 ,"_id": 0})
for p in products_names:
    print(p['product_name'], p['brand_name'])



4.9
['BrandA', 'BrandD', 'BrandE']
PhoneA BrandA
CameraD BrandD
HeadphonesE BrandE


In [ ]:
# Display customers who live in a specified city and have a bill amount greater than 50000.
specified_city = "CityB"
min_bill_amount = 50000

cid = orders.find({"bill_amount": {"$gt": min_bill_amount}})
# Find customers in the specified city with high bill amounts
for order in cid:
    customer = customers.find_one({"_id": order["customer_id"], "city": specified_city}, {"customer_name": 1, "_id": 0})
    
    if customer:
        print({"customer_name": customer["customer_name"], "bill_amount": order["bill_amount"]})


{'customer_name': 'Patricia Green', 'bill_amount': 60000}
